Imports

In [1]:
import mediapipe as mp
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler    
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# Load DATA

In [2]:
# The data labels are in the form below:
alphabet = ['1', '2', '3','4', 'space', 'del', 
'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
labels_ = alphabet
NUM_CLASSES = len(labels_)

In [3]:
x_flat = pd.read_csv("./data/train/x_data900.csv") 
y_flat = pd.read_csv("./data/train/y_data900.csv")  #Note the first column of this is the index, so it's nx2 size

x_flat_test = pd.read_csv("./data/test/x_data_test100.csv")
y_flat_test = pd.read_csv("./data/test/y_data_test100.csv") #Note the first column of this is the index, so it's nx2 size



In [4]:
# We need to unflatten
def unflatten(df,shape):
    """ 
    reshape a dataframe to the correct array size
    shape is an array ex [21,3] which describes the desired shape
    """
    old_array = df.to_numpy()
    new_array = []
    for point in old_array:
        point_reshaped = point.reshape(shape[0], shape[1])
        new_array.append(point_reshaped)

    return np.array(new_array)

# TEST
arr = np.array([[1,2,3,4,5,6,7,8],[1,2,3,4,5,6,7,8],[1,2,3,4,5,6,7,8]])
arr = pd.DataFrame(arr)
unflatten(arr,shape=[2,4])



array([[[1, 2, 3, 4],
        [5, 6, 7, 8]],

       [[1, 2, 3, 4],
        [5, 6, 7, 8]],

       [[1, 2, 3, 4],
        [5, 6, 7, 8]]])

In [5]:
#revert the data back into its original form
x_data = unflatten(x_flat,shape=[21,3])
X_test = unflatten(x_flat_test,shape=[21,3])

#Adjust labels to numpy array
y_data = y_flat.to_numpy()
y_test = y_flat_test.to_numpy()

print("training data shape:", np.shape(x_data), "training score's shape", np.shape(y_data))
print("testing data shape:", np.shape(X_test), "testing score's shape", np.shape(y_test))


training data shape: (24367, 21, 3) training score's shape (24367, 1)
testing data shape: (2659, 21, 3) testing score's shape (2659, 1)


In [6]:
# Validation split
X_train, X_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.1)

In [7]:
#One hot encode
y_train = to_categorical(y_train).astype(int)
y_valid = to_categorical(y_valid).astype(int)
y_test = to_categorical(y_test).astype(int)

Now we can train our models. 

# Modeling
Our inputs are:
- X_Train, y_train 
- X_valid, y_valid
- X_test,y_test
Each datapoint in the Xs are [21,3]

In [8]:
# Setup checkpoint
import os
import tensorflow as tf

checkpoint_path = "./checkpoints.ckpt/"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(21, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=NUM_CLASSES, activation='softmax'))

#Compile
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=5, callbacks=[cp_callback],validation_data=(X_valid, y_valid))

Epoch 1/5
667/686 [============================>.] - ETA: 0s - loss: 1.0454 - categorical_accuracy: 0.7656
Epoch 1: saving model to .\checkpoints.ckpt
686/686 [==============================] - 3s 4ms/step - loss: 1.0255 - categorical_accuracy: 0.7708 - val_loss: 0.2847 - val_categorical_accuracy: 0.9479
Epoch 2/5
672/686 [============================>.] - ETA: 0s - loss: 0.2088 - categorical_accuracy: 0.9695
Epoch 2: saving model to .\checkpoints.ckpt
686/686 [==============================] - 2s 3ms/step - loss: 0.2068 - categorical_accuracy: 0.9698 - val_loss: 0.1422 - val_categorical_accuracy: 0.9795
Epoch 3/5
683/686 [============================>.] - ETA: 0s - loss: 0.1367 - categorical_accuracy: 0.9801
Epoch 3: saving model to .\checkpoints.ckpt
686/686 [==============================] - 2s 3ms/step - loss: 0.1365 - categorical_accuracy: 0.9802 - val_loss: 0.1044 - val_categorical_accuracy: 0.9819
Epoch 4/5
666/686 [============================>.] - ETA: 0s - loss: 0.1028 - cate

In [11]:
# Evaluate on the test set:
score = model.evaluate(np.array(X_test), y_test, verbose = 0) 
print(f"Accuracy: {score[1]}")
print(f"Loss: {score[0]}")



Accuracy: 0.9845806956291199
Loss: 0.08085747808218002


# Video Demo
Now let's see how this performs live!

In [12]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        landmarks = []
        letter = ''
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
                
                # Get the bounding box of the hand
                x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
                for landmark in hand_landmarks.landmark:
                    x, y = landmark.x, landmark.y
                    x_min = min(x_min, x)
                    y_min = min(y_min, y)
                    x_max = max(x_max, x)
                    y_max = max(y_max, y)

                # Normalize the coordinates with respect to the bounding box of the hand
                for landmark in hand_landmarks.landmark:
                    x, y, z = landmark.x, landmark.y, landmark.z
                    x_norm = (x - x_min) / (x_max - x_min)
                    y_norm = (y - y_min) / (y_max - y_min)
                    landmarks.append([x_norm, y_norm, z])

            # Convert the landmarks to a feature vector
            x_t = np.array(landmarks)
                        
            
            if x_t.flatten().shape[0] == 63:
            
                res_ = model.predict(np.array([x_t]), verbose = 0)

                index_class = np.argmax(res_)

                letter = alphabet[index_class]


        # Flip the image horizontally for a selfie-view display.
        
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (100,50)
        fontScale              = 1
        fontColor              = (0,0,0)
        thickness              = 4
        lineType               = 2

        image = cv2.putText(cv2.flip(image, 1),letter, 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            thickness,
            lineType)
        
        
        cv2.imshow('MediaPipe Hands', image)
            
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
